# Textrepräsentation
Um Texte effizient mit Methoden der Data Science verarbeiten zu können, müssen diese in einen einheitlichen, numerisch berechenbaren Raum konvertiert werden. In diesem Abschnitt werden dazu zwei Methoden präsentiert, die Texte in Vektoren konvertieren, um diese anschließend durch Ähnlichkeitsmaße innerhalb eines Vektorraums zu vergleichen.

In [ ]:
import math
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stopwords_and_punctuation = set((*stopwords.words('german'), ',', '.', ';', ':', '(', ')', '!', '?', "'", '’'))

## Inhaltsverzeichnis
- [Beispieltexte](#Beispieltexte)
- [Bag-of-Words-Modell](#Bag-of-Words-Modell)
- [TF-IDF Modell](#TF-IDF-Modell)
- [Cosinus-Ähnlichkeit](#Cosinus-Ähnlichkeit)
- [Zusammenfassung](#Zusammenfassung)

## Beispieltexte
Im Folgenden verwenden wir einfache Beispieltexte zur Veranschaulichung:

In [ ]:
example_texts = [
    'Ich liebe Hunde, denn Hunde sind die tollsten Tiere.',
    'Ich liebe Katzen. Katzen sind einfach unabhängig und mutig.',
    'Ich liebe Katzen und Hunde.',
    'Ich liebe Delphine. Im Wasser lebende Säugetiere sind mythisch.'
]

## Bag-of-Words-Modell
Das Bag-of-Words-Modell basiert auf der Idee, dass der Inhalt eines Textes durch die Häufigkeit der Wörter im Text beschrieben werden kann. Beim Erstellen der Repräsentation eines Textes wird daher auch die Reihenfolge und Struktur der Wörter ignoriert und tatsächlich nur die Anzahl gezählt, wie oft jedes Wort im Dokument vorkommt.

Das Erstellen der Vektoren funktioniert nun in zwei Schritten. Zuerst wird ein sogenanntes *Vokabular* erstellt, indem jedem einzelnen Wort ein Position für den späteren Vektor zugeteilt wird. Das Vokabular für drei Sätze "Ich liebe Hunde.", "Ich liebe Katzen." und "Ich hasse Delphine." wäre nach Entfernung von Stoppwörtern beispielsweise

```python
['liebe', 'Hunde', 'Katzen', 'hasse', 'Delphine']
```

Die Reihenfolge an sich ist nicht entscheidend. Relevant ist nur, dass sie im weiteren Verlauf konstant bleibt.

Für die Implementierung in Python wird ein Dictionary verwendet, um später anhand eines Wortes schnell den Index zu finden:

In [ ]:
def bow_vocabulary(texts):
    voc = {}

    for text in texts:
        for token in word_tokenize(text.casefold()):
            if token not in stopwords_and_punctuation and token not in voc:
                voc[token] = len(voc)

    return voc


example_vocabulary = bow_vocabulary(example_texts)
example_vocabulary

Im zweiten Schritt wird jedem Text ein Vektor zugeordnet. Dieser Vektor basiert auf dem Vokabular und enthält die Häufigkeit des dieser Stelle des Vektors zugeordneten Wortes im Text.

Die oben genannten Textbeispiele führen daher zu folgenden Vektoren:
- "Ich liebe Hunde." $\rightarrow$ `[1, 1, 0, 0, 0]`
- "Ich liebe Katzen." $\rightarrow$ `[1, 0, 1, 0, 0]`
- "Ich hasse Delphine." $\rightarrow$ `[0, 0, 0, 1, 0]`

Mit Hilfe des erzeugten Dictionaries lassen sich solche Vektoren nun auch einfach in Python bilden:

In [ ]:
def bow_vector(vocabulary, text):
    vector = [0 for _ in vocabulary]

    for token in word_tokenize(text.casefold()):
        if token in vocabulary:
            vector[vocabulary[token]] += 1

    return vector


example_bow_vectors = [bow_vector(example_vocabulary, text) for text in example_texts]
example_bow_vectors

Wenn für jeden Text bzw. für jedes Dokument ein solcher Vektor erzeugt wurde, lassen sich diese durch Einsatz einer geeigneten Ähnlichkeitsfunktion vergleichen. Die folgende Zelle verwendet dafür die Cosinus-Ähnlichkeit, auf die wir später noch einmal zurückkommen. Für den Moment reicht es zu wissen, dass ein Wert zwischen $0$ und $1$ erzeugt wird, wobei $0$ für keine Ähnlichkeit und $1$ für maximale Ähnlichkeit steht.

In [ ]:
for text1, vector1 in zip(example_texts, example_bow_vectors):
    for text2, vector2 in zip(example_texts, example_bow_vectors):
        if text1 >= text2:
            continue

        np_vector1 = np.array(vector1)
        np_vector2 = np.array(vector2)

        print(text1)
        print(text2)
        print(np.dot(np_vector1, np_vector2) / (np.linalg.norm(np_vector1) * np.linalg.norm(np_vector2)))
        print()

Alternativ lässt sich das BoW-Modell auch in einer binären Form umsetzen. Dafür wird entweder $1$ im Vektor gespeichert, wenn der Term *mindestens* einmal vorkommt, oder $0$ andernfalls.

## TF-IDF Modell
Im Bag-of-Words-Modell wird die absolute Häufigkeit aller Wörter betrachtet und damit auch jedes Wort gleich gewichtet. Es ist aber davon auszugehen, dass ein Wort, das zehn Mal vorkommt, nicht zehn Mal so relevant innerhalb eines Textes ist wie ein Wort, das nur ein einziges Mal vorkommt. Zudem sollten Worte, die in relativ vielen der Texte vorkommen, weniger gewichtet werden, da sie keine Information repräsentieren, welche die Texte voneinander unterscheidbar macht.

Das TF-IDF Modell bezieht daher auch Dokument- und Korpusstatistiken mit ein. Dazu führen wir mehrere Häufigkeiten ein, die wir anschließend miteinander ins Verhältnis setzen:
- **Term Frequency** (Termhäufigkeit) $tf_{t,d}$ bezeichnet die Häufigkeit des Auftretens des Terms (Wortes) $t$ im Dokument (Text) $d$.
- **Document Frequency** (Dokumenthäufigkeit) $df_t$ bezeichnet die Anzahl der Dokumente, die den Term $t$ enthalten.
- **Inverse Document Frequence** (Inverse Dokumentenhäufigkeit) $idf_t$ bildet sich für jeden einzelnen Term $t$ als $$ idf_t = log\left( \frac{\left| D \right|}{df_t} \right) $$

Die TF-IDF Gewichtung $w_{t,d}$ für einen Term $t$ innerhalb eines Dokuments $d$ berechnet sich dann aus der Häufigkeit des Terms im Dokument und der inversen Dokumentenhäufigkeit des Terms: $$ w_{t,d} = tf_{t,d} * idf_t $$

Aber ineffiziente, aber einfach verständliche Umsetzung in Python könnte dann zum Beispiel wie folgt aussehen:

In [ ]:
def tf(term, document):
    count = 0

    for token in word_tokenize(document.casefold()):
            if token not in stopwords_and_punctuation and term == token:
                count += 1

    return count

def df(term, documents):
    count = 0

    for document in documents:
        if tf(term, document) > 0:
            count += 1

    return count

def idf(term, documents):
    return math.log(len(documents) / df(term, documents))

In [ ]:
def w(term, document, documents):
    return tf(term, document) * idf(term, documents)

In [ ]:
def tfidf_vector(vocabulary, document, documents):
    return [w(term, document, documents) for term in vocabulary]

example_tfidf_vectors = [tfidf_vector(example_vocabulary, text, example_texts) for text in example_texts]
example_tfidf_vectors

Auch diese Vektoren lassen sich zum Beispiel mithilfe der Cosinus-Ähnlichkeit vergleichen:

In [ ]:
for text1, vector1 in zip(example_texts, example_tfidf_vectors):
    for text2, vector2 in zip(example_texts, example_tfidf_vectors):
        if text1 >= text2:
            continue

        np_vector1 = np.array(vector1)
        np_vector2 = np.array(vector2)

        print(text1)
        print(text2)
        print(np.dot(np_vector1, np_vector2) / (np.linalg.norm(np_vector1) * np.linalg.norm(np_vector2)))
        print()

Es fällt dabei auf, dass die Ähnlichkeiten auf $0$ sinken, wenn von verschiedenen Tieren gesprochen wird. Der übereinstimmende Term *liebe* wird vom BoW-Modell regulär gezählt, während das TF-IDF Modell diesem Term keine Bedeutung zuordnet, da er in jedem einzelnen Dokument vorkommt.

## Cosinus-Ähnlichkeit
Die Länge eines Textes hat Einfluss auf den gebildeten Vektor. Beim BoW-Modell steigt mit der Anzahl der Terme auch die Summe über alle Elemente des Vektors, während beim TF-IDF Modell die Anzahl der Elemente abnimmt, die $0$ sind. Ein Vergleich zweier Vektoren sollte daher unabhängig von der Länge des Vektors sein.

Die Cosinus-Ähnlichkeit ist ein Maß für die Ähnlichkeit zwischen zwei Vektoren im n-dimensionalen Raum, das häufig in der Textanalyse und im maschinellen Lernen verwendet wird. Sie basiert auf dem Cosinus des Winkels zwischen den beiden Vektoren und gibt an, wie ähnlich diese zueinander ausgerichtet sind. Ein Wert von $1$ bedeutet, dass die Vektoren einander **überlagern** (also exakt in die gleiche Richtung weisen), während ein Wert von $0$ darauf schließen lässt, dass sie orthogonal zueinander stehen und somit maximal unähnlich sind. Die Länge des Vektors wird somit gar nicht einbezogen.

Der Cosinus des Winkels $\theta$ zwischen zwei Vektoren $v_1$ und $v_2$ berechnet sich wie folgt. Dabei bezeichnet $\cdot$ das Skalarprodukt und $\left| v \right|$ die Länge des Vektors $v$:
$$
cos(\theta) = \frac{v_1 \cdot v_2}{\left|v_1\right| * \left|v_2\right|}
$$

Die Umsetzung in Python ist mit Hilfe von NumPy sehr einfach möglich:

In [ ]:
def cos_sim(v1, v2):
    np_v1 = np.array(v1)
    np_v2 = np.array(v2)

    return np.dot(np_v1, np_v2) / (np.linalg.norm(np_v1) * np.linalg.norm(np_v2))

Anhand einiger Vektoren lässt sich die Funktion leicht nachvollziehen:

In [ ]:
print('unterschiedliche Länge, gleiche Richtung:  ', cos_sim([1, 1], [5, 5]))
print('unterschiedliche Länge, ähnliche Richtung: ', cos_sim([1, 1], [2, 1]))
print('gleiche Länge, orthogonal:                 ', cos_sim([1, 0], [0, 1]))
print('unterschiedliche Länge, fast orthogonal:   ', cos_sim([9, 0], [1, 5]))

## Zusammenfassung
Die Wahl der Repräsentationsform hat starken Einfluss darauf, welche Terme mit welcher Gewichtung in den Dokumentenvektor einfließen. Zur Untersuchung der Ähnlichkeit bietet sich die Cosinus-Ähnlichkeit an, welche die Länge der Vektoren ignoriert und stattdessen den Winkel zwischen diesen zum Vergleich heranzieht.